In [ ]:
import pandas as pd
import re

In [ ]:
# Настройки отображения датафреймов
pd.set_option('display.max_rows', None)  # Показывать все строки
pd.set_option('display.max_columns', None)  # Показывать все столбцы
pd.set_option('display.width', None)  # Автоматическая ширина
pd.set_option('display.max_colwidth', None)  # Показывать полное содержимое ячеек
pd.set_option('display.expand_frame_repr', True)  # Разрешить перенос строк

In [ ]:
class MarketImpactClassifier:
    def __init__(self):
        # Голубые фишки и их вариации названий
        self.blue_chips = [
            # Нефть и газ
            r'газпром', r'газпрома?', r'гп', r'(пао\s+)?газпром\s+нефть',
            r'лукойл', r'лукойла?', r'роснефть', r'роснефти', 
            r'новатэк', r'новатэка?', r'транснефть', r'транснефти',
            r'сургутнефтегаз', r'сургута?', r'татнефть', r'татнефти',
            
            # Банки и финансы
            r'сбербанк', r'сбера?', r'втб', r'тинькофф', r'тинькова?',
            r'московск[ауюия]{2,3}\s+бирж[ауия]', r'мосбирж[ауия]', r'моех',
            
            # Металлургия и добыча
            r'норникель', r'гмк', r'северсталь', r'северстали',
            r'полюс', r'полюса?', r'полиметалл', r'полиметалла?',
            r'алроса', r'алросы', r'нлмк', r'магнит', r'магнита?',
            r'русал', r'русала?', r'фосагро', r'фосагро',
            
            # Телекоммуникации
            r'мтс', r'ростелеком', r'ростелекома?',
            
            # Другие крупные компании
            r'яндекс', r'яндекса?', r'озон', r'озона?',
            r'аэрофлот', r'аэрофлота?', r'mail\.ru', r'vk'
        ]

        # Ключевые слова для высокого влияния
        self.high_impact_keywords = [
            # Рыночные термины и индикаторы
            r'акци[ияйюе]', r'котировк[иау]', r'бирж[ауе]', r'ммвб', r'индекс[ауы]?',
            r'рынок', r'рынк[ауе]', r'трейдинг', r'инвестор[ауы]?', r'инвестиц',
            r'капитализаци[яию]', r'волатильност[ьию]', r'ликвидност[ьию]',
            r'(торгов|биржев)[ыаяой]{2,3} сесси[яию]', r'ралли', r'коррекци[яию]',
            r'растущ[ийая]{2,3} тренд', r'нисходящ[ийая]{2,3} тренд', r'разворот',
            r'просадк[аиу]', r'рост', r'падени[еяю]', r'обвал', r'отскок',
            r'медвеж[ийая]{2,3}', r'бычь[ийая]{2,3}', r'стагнаци[яию]',
            r'ценн[ыеая]{2,3} бумаг[иа]', r'облигаци[ияю]', r'офз', r'евробонд[ыау]',
            r'фьючерс[ыау]?', r'опцион[ыау]?', r'дериватив[ыау]?',
            r'маржинальн[ыаяое]{2,3}', r'плечо', r'левередж',
            r'шорт[ыау]?', r'лонг[иау]?', r'позици[яию]',
            r'рыночн[ыаяое]{2,3} (риск|стоимост[ьи])',
            r'брокер[ауы]?', r'брокерск[ийая]{2,3}', r'депозитар[ийная]{2,3}',
            r'портфель', r'портфельн[ыеая]{2,3}', r'диверсификаци[яию]',
            r'спред[ыау]?', r'премаркет', r'постмаркет',
            r'(дневн|недельн|месячн)[ыоеая]{2,3} (максимум|минимум)',
            r'уровень поддержки', r'уровень сопротивления',
            r'стоп(-)?(лосс|лос)', r'тейк(-)?(профит|профит)',
            r'ликвидн[ыеая]{2,3}', r'голубы[еях] фишк[иау]',
            r'рыночн[ыаяое]{2,3} капитализаци[яию]',
            # Макроэкономика
            r'ключев[аяуюо]{2,3}\s+ставк[аиу]', r'инфляци[яию]', r'ввп', 
            r'центральный банк', r'центробанк', r'цб рф', r'банк россии',
            r'курс[ауе]? (доллар|евро|юан|валют)', r'минфин', r'минэкономразвития',
            r'ставк[аиу] по ипотеке', r'ставк[аиу] по кредит', r'девальвац',
            r'экономическ[а-я]{2,3} кризис', r'рецесси[яию]', r'дефолт',
            r'торгов[ыоего]{2,3} баланс', r'платежн[ыоего]{2,3} баланс',
            r'золотовалютн[ыоего]{2,3} резерв', r'фонд национального благосостояния',
            r'фнб', r'госдолг', r'государственн[ыоего]{2,3} долг',
            r'дефицит бюджета', r'профицит бюджета',
            
            # Геополитика и санкции
            r'санкци[ияю]', r'запрет', r'ограничени[еяю]', r'эмбарго',
            r'заморозк[аиу] (счет|актив)', r'конфискац', r'арест[а-я]* (счет|актив)',
            r'нерезидент', r'спецоперац', r'военн[а-я]{2,3} операц',
            r'боев[ыоего]{2,3} действ', r'мобилизац', r'контрсанкц',
            r'недружественн[ыоего]{2,3} стран[ыа]', r'импортозамещени[еяю]',
            r'параллельн[ыоего]{2,3} импорт',
            
            # Нефть, газ и энергетика
            r'нефт[ьи]', r'газ[ауе]?', r'brent', r'urals', 
            r'опек', r'опек\+', r'опек плюс', r'добыч[аиу] нефти',
            r'газопровод', r'северн[ыоего]{2,3} поток', r'турецк[ийого]{2,3} поток',
            r'сил[ауые] сибири', r'спг', r'сжиженн[ыоего]{2,3} газ',
            r'энергетическ[а-я]{2,3} кризис', r'энергоресурс',
            
            # Корпоративные события
            r'слияни[еяю]', r'поглощени[еяю]', r'ipo', r'листинг',
            r'делистинг', r'размещени[еяю]', r'buy\s*back', r'обратн[ыоего]{2,3} выкуп',
            r'дивиденд', r'отчетност[ьи]', r'прибыл[ьи]', r'убыт[окки]',
            r'(финансов|операцион)[а-я]{2,3} результат', r'ebitda',
            r'капитализац', r'free\s*float', r'дол[яию] рынка',
            
            # Регуляторы и законодательство
            r'правительств[оа]', r'госдум[аы]', r'законопроект',
            r'федеральн[ыоего]{2,3} закон', r'указ президента',
            r'постановлени[еяю]', r'законодательств[оа]', r'регулировани[еяю]',
            r'лицензи[яию]', r'налог', r'пошлин[аыу]', r'сбор[ыа]',
            r'акциз[ыа]', r'тариф', r'субсиди[яию]', r'господдержк[аиу]'
        ]
        
        # Ключевые слова для среднего влияния
        self.medium_impact_keywords = [
            # Отраслевые новости
            r'отрасл[ьи]', r'сектор[ауе]?', r'промышленност[ьи]', 
            r'производств[оа]', r'машиностроени[еяю]', r'легк[ауюо]{2,3} промышленност[ьи]',
            r'тяжел[ауюо]{2,3} промышленност[ьи]', r'строительств[оа]',
            r'девелопмент', r'недвижимост[ьи]', r'ретейл', r'розниц[ауе]',
            r'e-commerce', r'электронн[ауюо]{2,3} торговл[яию]',
            
            # Компании и бизнес
            r'компани[яию]', r'предприяти[еяю]', r'бизнес[ауе]?',
            r'малый бизнес', r'средний бизнес', r'мсп',
            r'стартап', r'инвестиц', r'инвестор',
            r'акционер', r'совет директоров',
            
            # Региональные новости
            r'регион[ауе]?', r'област[ьи]', r'республик[аи]', r'кра[йя]',
            r'округ[ауе]?', r'муниципалитет', r'агломерац',
            
            # Цифровизация и технологии
            r'цифровизац', r'импортозамещени[еяю]', r'технологи',
            r'инноваци', r'искусственн[ыоего]{2,3} интеллект',
            r'нейросет[ьи]', r'блокчейн', r'криптовалют',
            
            # Рынок труда
            r'безработиц[ауе]', r'занятост[ьи]', r'рынок труда',
            r'зарплат[ауе]', r'оплат[ауе] труда', r'вакансии'
        ]
        
        # Компиляция регулярных выражений
        self.blue_chips_patterns = [re.compile(pattern, re.IGNORECASE) for pattern in self.blue_chips]
        self.high_impact_patterns = [re.compile(pattern, re.IGNORECASE) for pattern in self.high_impact_keywords]
        self.medium_impact_patterns = [re.compile(pattern, re.IGNORECASE) for pattern in self.medium_impact_keywords]

    def classify_news(self, text):
        """
        Классифицирует новость по уровню потенциального влияния на рынок
        
        Returns:
        --------
        impact_level : str
            'HIGH', 'MEDIUM' или 'LOW'
        matched_keywords : list
            Список найденных ключевых слов
        """
        # Проверяем упоминания голубых фишек
        blue_chips_matches = []
        for pattern in self.blue_chips_patterns:
            matches = pattern.findall(text)
            if matches:
                # Если matches - список кортежей, берем первый элемент каждого кортежа
                if matches and isinstance(matches[0], tuple):
                    matches = [m[0] for m in matches]
                blue_chips_matches.extend(matches)
        
        # Проверяем на высокое влияние
        high_impact_matches = []
        for pattern in self.high_impact_patterns:
            matches = pattern.findall(text)
            if matches:
                # Если matches - список кортежей, берем первый элемент каждого кортежа
                if matches and isinstance(matches[0], tuple):
                    matches = [m[0] for m in matches]
                high_impact_matches.extend(matches)
        
        # Если есть упоминание голубых фишек или другие признаки высокого влияния
        if blue_chips_matches or high_impact_matches:
            all_matches = blue_chips_matches + high_impact_matches
            return 'HIGH', all_matches
        
        # Проверяем на среднее влияние
        medium_impact_matches = []
        for pattern in self.medium_impact_patterns:
            matches = pattern.findall(text)
            if matches:
                # Если matches - список кортежей, берем первый элемент каждого кортежа
                if matches and isinstance(matches[0], tuple):
                    matches = [m[0] for m in matches]
                medium_impact_matches.extend(matches)
        
        if medium_impact_matches:
            return 'MEDIUM', medium_impact_matches
        
        # Если ничего не найдено, считаем влияние низким
        return 'LOW', []

def process_news_file(filename):
    """Обрабатывает файл с новостями и классифицирует их"""
    # Читаем файл
    df = pd.read_csv(filename)
    
    # Создаем классификатор
    classifier = MarketImpactClassifier()
    
    # Классифицируем новости
    results = []
    for _, row in df.iterrows():
        impact_level, keywords = classifier.classify_news(row['text'])
        results.append({
            'date': row['date'],
            'time': row['time'],
            'text': row['text'],
            'impact_level': impact_level,
            'matched_keywords': ', '.join(set(keywords)) if keywords else None
        })
    
    # Создаем новый датафрейм с результатами
    results_df = pd.DataFrame(results)
    
    # Выводим начальную статистику
    print("\nНачальная статистика по уровням влияния:")
    print(results_df['impact_level'].value_counts())
    
    # Удаляем новости с низким уровнем влияния
    results_df = results_df[results_df['impact_level'] != 'LOW']
    
    # Выводим статистику после фильтрации
    print("\nСтатистика после удаления новостей с низким влиянием:")
    print(results_df['impact_level'].value_counts())
    
    # Выводим примеры для оставшихся категорий
    for level in ['HIGH', 'MEDIUM']:
        print(f"\nПримеры новостей с уровнем влияния {level}:")
        examples = results_df[results_df['impact_level'] == level].iloc[:2]
        for _, row in examples.iterrows():
            print(f"\n[{row['date']} {row['time']}]")
            if row['matched_keywords']:
                print(f"Ключевые слова: {row['matched_keywords']}")
            print(row['text'])
    
    # Сохраняем результаты
    output_filename = 'market_impact_news.csv'
    results_df.to_csv(output_filename, index=False)
    print(f"\nРезультаты сохранены в {output_filename}")
    
    return results_df

In [ ]:
if __name__ == "__main__":
    results_df = process_news_file('../merged_df/news_tg.csv')

In [ ]:
hh = pd.read_csv('market_impact_news.csv')
hh.head()

In [ ]:
hh[-15:]